In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%load_ext dotenv

In [3]:
import os

In [4]:
!pwd
!ls -l ./

/mnt/datadrive01/prj/AzureChestXRayNoAML/code
total 572
-rwxrwxrwx 1 loginVM_001 loginVM_001  12207 Sep 24 00:53 001_get_data.ipynb
-rwxrwxrwx 1 loginVM_001 loginVM_001 265507 Sep 24 00:44 00_create_docker_image.html
-rwxrwxrwx 1 loginVM_001 loginVM_001  10414 Sep 24 00:58 00_create_docker_image.ipynb
drwxr-xr-x 3        1000 users         4096 Sep 23 23:12 chestxray
drwxr-xr-x 3        1000 users         4096 Sep 23 23:12 code
-rw-r--r-- 1        1000 users       255403 Sep 23 23:56 edit_python_files.html
-rw-r--r-- 1        1000 users         6602 Sep 23 23:58 edit_python_files.ipynb
drwxr-xr-x 3        1000 users         4096 Sep 24 00:22 notShared
-rw-r--r-- 1        1000 users         2452 Nov 27  2017 packages-microsoft-prod.deb
-rw-r--r-- 1        1000 users         2452 Nov 27  2017 packages-microsoft-prod.deb.1
drwxrwxrwx 3 loginVM_001 loginVM_001   4096 Sep 23 23:10 src
drwxr-xr-x 2        1000 users         4096 Sep 24 00:27 tmp


In [5]:
%%writefile .env
# This cell is tagged `parameters`
# Please modify the values below as you see fit

# Your docker login and image repository name
docker_login = 'georgedockeraccount'
image_tag = "/chestxray-no-aml-gpu"

Overwriting .env


In [6]:
%%writefile ./../docker/dockerfile

FROM nvidia/cuda:9.0-cudnn7-devel
#FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04

MAINTAINER George Iordanescu <ghiordan@microsoft.com>
#based on https://github.com/keras-team/keras/blob/master/docker/Dockerfile

# Install system packages
RUN apt-get update && apt-get install -y --no-install-recommends \
      apt-utils \
      bzip2 \
      g++ \
      git \
      graphviz \
      libgl1-mesa-glx \
      libhdf5-dev \
      openmpi-bin \
      wget \
      ssh \
      rsync && \
    rm -rf /var/lib/apt/lists/*

# Install conda
ENV CONDA_DIR /opt/conda
ENV PATH $CONDA_DIR/bin:$PATH

RUN wget --quiet --no-check-certificate https://repo.continuum.io/miniconda/Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo "c59b3dd3cad550ac7596e0d599b91e75d88826db132e4146030ef471bb434e9a *Miniconda3-4.2.12-Linux-x86_64.sh" | sha256sum -c - && \
    /bin/bash /Miniconda3-4.2.12-Linux-x86_64.sh -f -b -p $CONDA_DIR && \
    rm Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo export PATH=$CONDA_DIR/bin:'$PATH' > /etc/profile.d/conda.sh 
        
#https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux
# https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy
# https://www.microsoft.com/net/download/linux-package-manager/ubuntu16-04/runtime-2.1.2
RUN wget -q https://packages.microsoft.com/config/ubuntu/16.04/packages-microsoft-prod.deb && \
    dpkg -i packages-microsoft-prod.deb && \ 
    apt-get install  -y --no-install-recommends apt-transport-https && \
    apt-get update && \
    apt-get install  -y --no-install-recommends aspnetcore-runtime-2.1  && \
    mkdir /tmp/azcopy && \
    wget -O /tmp/azcopy/azcopy.tar.gz https://aka.ms/downloadazcopyprlinux &&  \
    tar -xf /tmp/azcopy/azcopy.tar.gz -C /tmp/azcopy &&  \
    /tmp/azcopy/install.sh && \
    rm -rf /tmp/azcopy

# Install Python packages and keras
ENV NB_USER keras
ENV NB_UID 1000

RUN useradd -m -s /bin/bash -N -u $NB_UID $NB_USER && \
    chown $NB_USER $CONDA_DIR -R && \
    mkdir -p /src && \
    chown $NB_USER /src

USER $NB_USER

ARG python_version=3.6

# tensorflow-gpu 1.10.1 has requirement numpy<=1.14.5,>=1.13.3, but you'll have numpy 1.15.2 which is incompatible.
RUN conda install -y python=${python_version} && \
    pip install --upgrade pip && \
    pip install \
      sklearn_pandas \
      scikit-image \
      tensorflow-gpu \
      tensorflow-tensorboard \
      matplotlib \
      numpy && \
    conda install \
      bcolz \
      h5py \
      matplotlib \
      mkl \
      nose \
      notebook \
      opencv \
      Pillow \
      pandas \
      pydot \
      pygpu \
      pyyaml \
      requests \
      scikit-learn \
      six \
      tqdm && \
    git clone git://github.com/keras-team/keras.git /src && pip install -e /src[tests] && \
    pip install git+git://github.com/keras-team/keras.git && \
    pip install git+https://github.com/aleju/imgaug && \
    pip install git+https://www.github.com/keras-team/keras-contrib.git && \
    pip install http://download.pytorch.org/whl/cu90/torch-0.4.0-cp36-cp36m-linux_x86_64.whl \
      torchvision && \
    conda clean -yt     

#ADD theanorc /home/keras/.theanorc

ENV PYTHONPATH='/src/:$PYTHONPATH'

WORKDIR /src

EXPOSE 8888

# CMD jupyter notebook --port=8888 --ip=0.0.0.0


Overwriting ./../docker/dockerfile


In [7]:
%dotenv
docker_image_name = os.getenv('docker_login') + os.getenv('image_tag') + ':1.0.0'
docker_file_location = os.path.join(*(['.', '..','docker', 'dockerfile']))
working_path = '.'

docker_image_name
docker_file_location

'georgedockeraccount/chestxray-no-aml-gpu:1.0.0'

'./../docker/dockerfile'

In [8]:
!echo docker build -t $docker_image_name -f $docker_file_location $working_path
!docker build -t $docker_image_name -f $docker_file_location $working_path

docker build -t georgedockeraccount/chestxray-no-aml-gpu:1.0.0 -f ./../docker/dockerfile .
Sending build context to Docker daemon  612.9kB
Step 1/16 : FROM nvidia/cuda:9.0-cudnn7-devel
 ---> 288866c2c06c
Step 2/16 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Running in f0520b9236cd
Removing intermediate container f0520b9236cd
 ---> bc9e7c9ac9dd
Step 3/16 : RUN apt-get update && apt-get install -y --no-install-recommends       apt-utils       bzip2       g++       git       graphviz       libgl1-mesa-glx       libhdf5-dev       openmpi-bin       wget       ssh       rsync &&     rm -rf /var/lib/apt/lists/*
 ---> Running in 02513cc3ce43
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu xenial-security InRelease [107 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu xenial InRelease [247 kB]
Get:

Get:19 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libjpeg-turbo8 amd64 1.4.2-0ubuntu3.1 [111 kB]
Get:20 http://archive.ubuntu.com/ubuntu xenial/main amd64 libsm6 amd64 2:1.2.2-1 [15.8 kB]
Get:21 http://archive.ubuntu.com/ubuntu xenial/main amd64 libwrap0 amd64 7.6.q-25 [46.2 kB]
Get:22 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxdamage1 amd64 1:1.1.4-2 [6946 B]
Get:23 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxfixes3 amd64 1:5.0.1-2 [11.1 kB]
Get:24 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxshmfence1 amd64 1.2-1 [5042 B]
Get:25 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxxf86vm1 amd64 1:1.1.4-1 [10.6 kB]
Get:26 http://archive.ubuntu.com/ubuntu xenial/main amd64 libbsd0 amd64 0.8.2-1 [41.7 kB]
Get:27 http://archive.ubuntu.com/ubuntu xenial/main amd64 libedit2 amd64 3.1-20150325-1ubuntu2 [76.5 kB]
Get:28 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 openssh-client amd64 1:7.2p2-4ubuntu2.4 [589 kB]
Get:29 http://a

Get:100 http://archive.ubuntu.com/ubuntu xenial/universe amd64 libhwloc5 amd64 1.11.2-3 [99.5 kB]
Get:101 http://archive.ubuntu.com/ubuntu xenial/main amd64 libibverbs1 amd64 1.1.8-1.1ubuntu2 [25.0 kB]
Get:102 http://archive.ubuntu.com/ubuntu xenial/universe amd64 libopenmpi1.10 amd64 1.10.2-8ubuntu1 [2025 kB]
Get:103 http://archive.ubuntu.com/ubuntu xenial/universe amd64 openmpi-common all 1.10.2-8ubuntu1 [129 kB]
Get:104 http://archive.ubuntu.com/ubuntu xenial/universe amd64 openmpi-bin amd64 1.10.2-8ubuntu1 [100 kB]
debconf: delaying package configuration, since apt-utils is not installed
Fetched 44.5 MB in 4s (9076 kB/s)
Selecting previously unselected package libpopt0:amd64.
(Reading database ... 11603 files and directories currently installed.)
Preparing to unpack .../libpopt0_1.16-10_amd64.deb ...
Unpacking libpopt0:amd64 (1.16-10) ...
Selecting previously unselected package libglib2.0-0:amd64.
Preparing to unpack .../libglib2.0-0_2.48.2-0ubuntu4.1_amd64.deb ...
Unpacking libgli

Unpacking libjpeg8:amd64 (8c-2ubuntu8) ...
Selecting previously unselected package libtiff5:amd64.
Preparing to unpack .../libtiff5_4.0.6-1ubuntu0.4_amd64.deb ...
Unpacking libtiff5:amd64 (4.0.6-1ubuntu0.4) ...
Selecting previously unselected package libvpx3:amd64.
Preparing to unpack .../libvpx3_1.5.0-2ubuntu1_amd64.deb ...
Unpacking libvpx3:amd64 (1.5.0-2ubuntu1) ...
Selecting previously unselected package libxpm4:amd64.
Preparing to unpack .../libxpm4_1%3a3.5.11-1ubuntu0.16.04.1_amd64.deb ...
Unpacking libxpm4:amd64 (1:3.5.11-1ubuntu0.16.04.1) ...
Selecting previously unselected package libgd3:amd64.
Preparing to unpack .../libgd3_2.1.1-4ubuntu0.16.04.10_amd64.deb ...
Unpacking libgd3:amd64 (2.1.1-4ubuntu0.16.04.10) ...
Selecting previously unselected package libpixman-1-0:amd64.
Preparing to unpack .../libpixman-1-0_0.33.6-1_amd64.deb ...
Unpacking libpixman-1-0:amd64 (0.33.6-1) ...
Selecting previously unselected package libxcb-render0:amd64.
Preparing to unpack .../libxcb-render0

Selecting previously unselected package zlib1g-dev:amd64.
Preparing to unpack .../zlib1g-dev_1%3a1.2.8.dfsg-2ubuntu4.1_amd64.deb ...
Unpacking zlib1g-dev:amd64 (1:1.2.8.dfsg-2ubuntu4.1) ...
Selecting previously unselected package libjpeg-turbo8-dev:amd64.
Preparing to unpack .../libjpeg-turbo8-dev_1.4.2-0ubuntu3.1_amd64.deb ...
Unpacking libjpeg-turbo8-dev:amd64 (1.4.2-0ubuntu3.1) ...
Selecting previously unselected package libjpeg8-dev:amd64.
Preparing to unpack .../libjpeg8-dev_8c-2ubuntu8_amd64.deb ...
Unpacking libjpeg8-dev:amd64 (8c-2ubuntu8) ...
Selecting previously unselected package libjpeg-dev:amd64.
Preparing to unpack .../libjpeg-dev_8c-2ubuntu8_amd64.deb ...
Unpacking libjpeg-dev:amd64 (8c-2ubuntu8) ...
Selecting previously unselected package libaec-dev:amd64.
Preparing to unpack .../libaec-dev_0.3.2-1_amd64.deb ...
Unpacking libaec-dev:amd64 (0.3.2-1) ...
Selecting previously unselected package libhdf5-dev.
Preparing to unpack .../libhdf5-dev_1.8.16+docs-4ubuntu1.1_amd64.d

...

cairo-1.14.12- 100% |###############################| Time: 0:00:00  54.29 MB/s
certifi-2018.8 100% |###############################| Time: 0:00:00  39.92 MB/s
chardet-3.0.4- 100% |###############################| Time: 0:00:00  60.63 MB/s
click-6.7-py36 100% |###############################| Time: 0:00:00  58.42 MB/s
cloudpickle-0. 100% |###############################| Time: 0:00:00  40.15 MB/s
constantly-15. 100% |###############################| Time: 0:00:00  31.63 MB/s
dask-core-0.19 100% |###############################| Time: 0:00:00  55.26 MB/s
decorator-4.3. 100% |###############################| Time: 0:00:00  34.63 MB/s
entrypoints-0. 100% |###############################| Time: 0:00:00  19.73 MB/s
heapdict-1.0.0 100% |###############################| Time: 0:00:00  18.33 MB/s
incremental-17 100% |###############################| Time: 0:00:00  38.08 MB/s
ipython_genuti 100% |###############################| Time: 0:00:00  41.83 MB/s
kiwisolver-1.0 100% |###################

  Running setup.py bdist_wheel for pytest-pep8: started
  Running setup.py bdist_wheel for pytest-pep8: finished with status 'done'
  Stored in directory: /home/keras/.cache/pip/wheels/68/34/52/2a6e08e030cef8b3671898f34cecb12750cc4b96ebfe66fe67
  Running setup.py bdist_wheel for pytest-cache: started
  Running setup.py bdist_wheel for pytest-cache: finished with status 'done'
  Stored in directory: /home/keras/.cache/pip/wheels/e4/9e/28/59d0a23800e49808f17920c5922252c661966ca4a36db5ee38
Successfully built pytest-pep8 pytest-cache
  Running setup.py develop for Keras
  Cloning git://github.com/keras-team/keras.git to /tmp/pip-req-build-afel7u_w
  Running setup.py bdist_wheel for Keras: started
  Running setup.py bdist_wheel for Keras: finished with status 'done'
  Stored in directory: /tmp/pip-ephem-wheel-cache-flmisrek/wheels/e6/02/ad/5e8e1a5824af71082e2260fe7e2eaa1b745c34706e6ff0a14b
Successfully built Keras
  Cloning https://github.com/aleju/imgaug to /tmp/pip-req-build-paqxtli6


  Running setup.py bdist_wheel for imgaug: started
  Running setup.py bdist_wheel for imgaug: finished with status 'done'
  Stored in directory: /tmp/pip-ephem-wheel-cache-xqz2fy4z/wheels/9c/f6/aa/41dcf2f29cc1de1da4ad840ef5393514bead64ac9e644260ff
Successfully built imgaug
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-mpdaavd8
  Running setup.py bdist_wheel for keras-contrib: started
  Running setup.py bdist_wheel for keras-contrib: finished with status 'done'
  Stored in directory: /tmp/pip-ephem-wheel-cache-_veyg8eu/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
Cache location: /opt/conda/pkgs
Will remove the following tarballs:

/opt/conda/pkgs
---------------
asn1crypto-0.22.0-py36_0.tar.bz2             148 KB
snappy-1.1.7-hbae5bb6_3.tar.bz2               38 KB
cloudpickle-0.5.5-py36_0.tar.bz2              26 KB
wheel-0.29.0-py36_0.tar.bz2                   88 KB
jpeg-9b-h024ee3a_2.tar.bz2   

pixman-0.34.0-hceecf20_3.tar.bz2             598 KB
harfbuzz-1.8.8-hffaf4a1_0.tar.bz2            863 KB
pexpect-4.6.0-py36_0.tar.bz2                  77 KB
tk-8.6.8-hbc83047_0.tar.bz2                  3.1 MB
libgfortran-ng-7.3.0-hdf63c60_0.tar.bz2      1.3 MB
bleach-2.1.4-py36_0.tar.bz2                   33 KB
pyzmq-17.1.2-py36h14c3975_0.tar.bz2          454 KB
jsonschema-2.6.0-py36_0.tar.bz2               62 KB
automat-0.7.0-py36_0.tar.bz2                  52 KB
cffi-1.10.0-py36_0.tar.bz2                   341 KB
notebook-5.6.0-py36_0.tar.bz2                7.4 MB
bzip2-1.0.6-h14c3975_5.tar.bz2               414 KB
ipykernel-4.9.0-py36_1.tar.bz2               146 KB
scikit-learn-0.19.2-py36h4989274_0.tar.bz2     5.2 MB
pyasn1-modules-0.2.2-py36_0.tar.bz2           86 KB
cytoolz-0.9.0.1-py36h14c3975_1.tar.bz2       419 KB
freetype-2.9.1-h8a8886c_1.tar.bz2            822 KB
libstdcxx-ng-8.2.0-hdf63c60_1.tar.bz2        2.9 MB

---------------------------------------------------
Total:   

Removing intermediate container 459ce5e55ca3
 ---> 278df7f34221
Step 14/16 : ENV PYTHONPATH='/src/:$PYTHONPATH'
 ---> Running in 41a1d1fef285
Removing intermediate container 41a1d1fef285
 ---> 814bd7080f5c
Step 15/16 : WORKDIR /src
Removing intermediate container 495f9d466d99
 ---> f951153d2f3c
Step 16/16 : EXPOSE 8888
 ---> Running in 1ff5a01f4e68
Removing intermediate container 1ff5a01f4e68
 ---> 2846cca0ae3b
Successfully built 2846cca0ae3b
Successfully tagged georgedockeraccount/chestxray-no-aml-gpu:1.0.0


In [9]:
!docker push $docker_image_name

The push refers to repository [docker.io/georgedockeraccount/chestxray-no-aml-gpu]

cc06d476: Preparing 
a89b6996: Preparing 
a1f3e5d0: Preparing 
9618113d: Preparing 
e517b5f3: Preparing 
e5d9d370: Preparing 
6d4f5f2e: Preparing 
3c790502: Preparing 
ac05da10: Preparing 
19f38a85: Preparing 
d6639b96: Preparing 
9e19929c: Preparing 
b2f378bb: Preparing 
afdbe580: Preparing 
43c86cbc: Preparing 


c06d476: Pushing  1.456GB/5.488GBPushing  243.3MB/5.488GBPushing  667.1MB/5.488GB

c06d476: Pushing  3.247GB/5.488GBPushing  1.948GB/5.488GBPushing  2.524GB/5.488GBPushing  2.583GB/5.488GBPushing  3.178GB/5.488GB

c06d476: Pushing  5.125GB/5.488GBPushing  3.928GB/5.488GBPushing   4.43GB/5.488GBPushing   4.64GB/5.488GBPushing  4.919GB/5.488GB

c06d476: Pushed   5.579GB/5.488GBPushing  5.146GB/5.488GB1.0.0: digest: sha256:97c206a15d7813782b89f3092c78be43f13d99a4f7f7c7be88bc40a16a7c5cbd size: 3688


In [19]:
!echo sudo nvidia-docker run -i -t -p 10003:8888 -v '$(pwd)':/local_dir:rw georgedockeraccount/chestxray-no-aml-gpu:1.0.0 /bin/bash -c '"/opt/conda/bin/jupyter notebook --notebook-dir=/local_dir --ip='*' --port=8888 --no-browser --allow-root"'

sudo nvidia-docker run -i -t -p 10003:8888 -v $(pwd):/local_dir:rw georgedockeraccount/chestxray-no-aml-gpu:1.0.0 /bin/bash -c "/opt/conda/bin/jupyter notebook --notebook-dir=/local_dir --ip=* --port=8888 --no-browser --allow-root"


In [21]:
!jupyter nbconvert --to html 00_create_docker_image.ipynb

[NbConvertApp] Converting notebook 00_create_docker_image.ipynb to html
[NbConvertApp] Writing 401244 bytes to 00_create_docker_image.html
